In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
class Parent:

    def __init__(self, other=None, name=None):  # only one constructor can be defined, need to treat all cases in it
        if other:
            self._name = other._name
            self._num = 1
            return
        if not name:
            name = 'noname'
        self._name = name

    def print_name(self):
        print(self._name)


# child will inherit all parent fields, and all methods to work with them, unless they're overriden
class Child(Parent):  
    def print_name_caps(self):
        print(self._name.capitalize())

In [2]:
p = Parent(name='Alex')
p.print_name()


p = Parent(p)
p.print_name()

Alex
Alex


In [5]:
c = Child(p)  # since copy constructor is defined in Parent - can create child from Parent 

c._num

c.print_name()
c.print_name_caps()

1

Alex
Alex


In [15]:
class Child(Parent):  

    def __init__(self, other=None, name=None):
        super().__init__(other, name)  # use base class constructor
        self._num = 2  # complement it with child class code


c = Child(p)
c._num

2

super() is preferred to use inside child classes. In the above example ```super().__init__``` would be equivalent to ```Parent.__init__```

In [17]:
issubclass(Child, Parent)  # use to check for subclass

True

composition is always recommeneded instead of inheritance if possible
* inheritance may create obscure construction, hard to work with
* existing code may check for type, not just use methods of a class, so inheritance won't guarantee that all code used for base class will work for the derived class
* instead of inheriting to just add a couple of methods and try to use derived class everywhere instead of base - just write several simple functions (instead of methods) to apply where necessary
* try to restrict inheritance to subclassing simple interface (e.g. via abc module), and not existing class that's used in many places already

* for each project list first all possible way to do it without inheritance

### ABC module

* collections have some classes derived from ABCs, of they can be further subclassed
* collections.abc submodule has some abstract classes to test different properties (e.g. if an object is hashable or not)

In [19]:
from abc import ABC

class MyABC(ABC):  # use ABC to define other ABCs
    pass

MyABC.register(tuple)  # can register any existing unrelated class and subclass (even built-in), called virtual subclasses

assert issubclass(tuple, MyABC)
assert isinstance((), MyABC)

tuple

In [35]:
# @abstractmethod decorator says that a method is abstract and must be overriden in all subclasses
# will work only for ABC subclass or ancestor
# must be the most inner if there are multiple decorators for a method
from abc import abstractmethod

class C(ABC):

    def __init__(self, x):
        self._x = x
    
    @property
    @abstractmethod  # abstract methods may have an implementation (unlike say Java)
    def x(self):
        return self._x
    
    @x.setter
    @abstractmethod
    def x(self, value):
        self._x = value 

    @abstractmethod
    def print_x(self):  # use pass to skip the base class definition of an abstract method 
        pass

# can use together with @classmethod, @staticmethod etc

In [34]:
# c = C(5)  # won't work, even though methods are defined - they are abstract

# concrete implementation of C
class D(C):

    @property
    def x(self):
        return super().x  # can call super() to use base class abstract method implementation
    
    @x.setter
    def x(self, value):
        super().x(self, value)

    def print_x(self):  # use pass to skip the base class definition of an abstract method 
        print(self.x)

d = D(5)
d.print_x()

5


In [50]:
# example
class C:
    
    def __init__(self, x):
        self._x = x

    def foo(self):
        return self._x

    def bar(self):
        return self.foo() + 10


class D(C):
    def foo(self):
        return self._x**2

d = D(5)
d.foo()
d.bar()  # bar() is not defined in D but it will correctly use the foo() method from D, not from C

25

35

## collections.abc

* ABCs in collections.abc provide ways to implement commonly needed interfaces, and subclass built-in class through composition + interface inheritance rather than direct subclassing
* examples below also provide good ways to use composition (field with 'base' class inside and passing methods to it, that are not overriden) rather than inheritance (and of course rather than multiple inheritance)

In [59]:
# subclassing built-in types are most subtle since a lot of methods are implemented in C and won't use other python methods

class MyDict(dict):
    def __getitem__(self, key):
        value = super().__getitem__(key)
        return set(value)

d = MyDict(a=[1, 2, 3])
d['a']  # works for get item

for k, v in d.items():
    print(k, v)  # overriding not applied here, actually .item() is not using __getitem__

# in such way it's not possible to implement only the key methods and have others implied from compositions

{1, 2, 3}

a [1, 2, 3]


In [76]:
# subclassing built-in classes is not recommended
# collections.abc contain abstract classes that requires a minimal set of method and will extend the other through compositions
# they replicate the behaviour of common built-in types


# example
from collections.abc import MutableMapping

# MutableMapping requires abstract method implementation for __getitem__, __setitem__, __delitem__, __len__, insert
# all other method will be implied

# can also be used to check objects for the implementation of the class behaviour
isinstance({'a': 1}, MutableMapping)


class MyDict(MutableMapping):   # using composition + collection.abc to 'subclass' built-in dict 
    def __init__(self, *args, **kwargs):  # pass constructor to dict()
        self._storage = dict(*args, **kwargs)

    def __setitem__(self, key, value):  # just pass the method to the internal base class variable (for those that we don't want to adjust)
        self._storage.__setitem__(key, value)

    def __getitem__(self, key):  # reimplement the method that we want to adjust
        return set(self._storage.__getitem__(key))

    def __delitem__(self, key):
        self._storage.__delitem__(key)

    def __iter__(self):
        return self._storage.__iter__()
    
    def __len__(self):
        return self._storage.__len__()

    # the methods above were compulsory to implement for MutableMapping (we used the dict implementation for most of them, except a custom implementation for __getitem__)

    def my_print(self):   # can add new methods
        print(self.keys(), self.values())  # keys() and values() already implied by MutableMapping


d = MyDict(a=[1, 2, 3])
d['a']

for k, v in d.items():  # works with modified getitem
    print(k, v)

True

{1, 2, 3}

a {1, 2, 3}


Attention: old ABCs from collections (like UserDict) are deprecated and must not be used. Use only ABCs from collections.abc

In [86]:
# extending built-in list with set intersection operators
from collections.abc import Sequence
# Sequence must implement __getitem__ and __len__

class MyList(Sequence):

    def __init__(self, *args, **kwargs):  # passing contruction to list() and store in a private field
        self._storage = list(*args, **kwargs)
    
    def __len__(self):
        return self._storage.__len__()

    def __getitem__(self, i):  # abstract Sequence methods are implemented (passed to list)
        return self._storage.__getitem__(i)

    def __and__(self, other):  # new set intersection operator
        return list(set(self._storage) & set(other))
    
    def __rand__(self, other):  # RHS operator version (to do e.g. [1, 2, 3] & my_list)
        return self.__and__(other)

lst = MyList([1, 2, 3])
lst[1]
lst & [2, 3, 4]

# usual sequence method (implied by Sequence interface)
1 in lst  # __contains__
for x in lst:  # __iter__
    print(x)

2

[2, 3]

True

1
2
3


In [94]:
# class attributes

class ExampleClass:

    class_attr = 0  # class attribute : shared across all class instances

    def __init__(self):
        self.instance_attr = 1

    @classmethod
    def change_attr(cls, new_attr_value):  # @classmethod is shared for all instance and can change class attributes
        cls.class_attr = new_attr_value

    @staticmethod
    def print_const():  # static method has the same definition as in C++ (shared across all instances)
        print('hello')


c = ExampleClass()
c.change_attr(5)  # change class attribute, can call using instance
c.class_attr

b = ExampleClass()
b.class_attr
b.change_attr(3)  # change via another instance
c.class_attr  # this will change class attribute globally

ExampleClass.change_attr(-2)  # can use class name
b.class_attr

b.print_const()
ExampleClass.print_const()  # calling static method works both from instances and class name

# always better to use class name for class and static methods, this is logically more correct

5

5

3

-2

hello
hello
